Before running the code, several dependant package need to be installed, skip if it already exists.

In [1]:
pip install urllib3

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pdfminer.six

     |████████████████████████████████| 5.6MB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install python-Levenshtein

Firstly, we make connection with database with pymysql api.

In [191]:
import pymysql
#https://cloud.tencent.com/developer/article/1395339
import urllib
import pdfminer
import copy
from io import StringIO
from io import BytesIO
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import *
from pdfminer.converter import PDFPageAggregator
from urllib.request import Request
from urllib.request import urlopen
#input basic info about database
class Processing():
    def __init__(self):
        self.NUM=3
        self.conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')

    def connection(self):
        query_result=[]
        #cursor is the current point we focus on, which could execute sql command.
        cursor=self.conn.cursor()
        #Here rw is the databse name and training_view is a view for requesting data. The execution should be around 30s.
        cursor.execute("SELECT * FROM krw.training_paper_view group by id limit %s;",self.NUM)
        for r in cursor:
            query_result.append(list(r))
        cursor.close()
        return query_result
    #lst
    def pre_processing(self,index,query_result):
        resourceId=query_result[index][2]
        authorNum=query_result[index][3]
        url=query_result[index][7]
        #print(resourceId,url)
        #name=lst[NUM][6]
        webList=["acmweb","aclweb"]
        for web in webList:
            if web in url:
                #pattern = re.compile(ur'^((https|http|ftp|rtsp|mms)?:\/\/)[^\s]+/')
                #str = u''
                url=url+".pdf"
            #name=pattern.search(str)
        #url="https://arxiv.org/pdf/"+str(name)+".pdf"
        print(resourceId,url)
        return resourceId,url
        
    def query(self,index, resourceId):
        temp=[]
        authorList=[]
        #Here krw is the databse name and training_view is a view for requesting data. The execution should be around 30s.
        #conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
        cursor=self.conn.cursor()
        #Here krw is the databse name and training_view is a view for requesting data. The execution should be around 30s.
        cursor.execute("SELECT * FROM krw.training_author_view where paper_id=(%s) group by name;",resourceId)
        cursor.close()
        authorPaper={}
        authorCitation={}
        authorID={}
        for r in cursor:
            temp.append(list(r))
        #print(temp)
        for line in temp:
            authorList.append(line[1])
            authorPaper[line[1]]=[line[3]]
            authorCitation[line[1]]=[line[4]]
            authorID[line[1]]=[line[0]]
        #print(authorList)
        return authorList,authorPaper,authorCitation,authorID

    def onlinePdfToTxt(self,dataIo,new_path):
        # Create PDF Parser
        parser = PDFParser(dataIo)
        # Create PDFDocument
        document = PDFDocument(parser)
        # Is it okay for extraction?
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        else:
            # Create PDF Manager
            resmag =PDFResourceManager()
            # Setting parameters
            laparams=LAParams()
            # Createing PDF device
            # device=PDFDevice(resmag )
            device=PDFPageAggregator(resmag ,laparams=laparams)
            # Create PDF explainer
            interpreter=PDFPageInterpreter(resmag ,device)
            # For each page
            for page in PDFPage.create_pages(document):
                interpreter.process_page(page)
                # accept this page's LTP object
                layout=device.get_result()
                for y in layout:
                    try:
                        if(isinstance(y,LTTextBoxHorizontal)):
                            with open('%s'%(new_path),'a',encoding="utf-8") as f:
                                f.write(y.get_text()+'\n')
                                #print("Success！")
                    except:
                        print("Failed")
                        
    def main(self,index):
        query_result=self.connection()
#         for index in range(len(NUM)):
        resourceId,url=self.pre_processing(index,query_result)
        html = urllib.request.urlopen(urllib.request.Request(url)).read()
        dataIo = BytesIO(html)
        self.onlinePdfToTxt(dataIo,'txt/'+str(resourceId)+'.txt')
        authorList,authorPaper,authorCitation,authorID=self.query(index,resourceId)
        return resourceId,authorList,authorPaper,authorCitation,authorID
    
# processing=Processing()
# processing.NUM=5
# resourceId,authorList,authorPaper,authorCitation,authorID=processing.main(4)
# print(authorList)

After excution we obtained a 2-dimensional matrix with each row represents a paper. Next, we constract a url based on arXiv id.

By following command we could access pdf by url and convert pdf 2 txt file and store at local.

In [70]:
import spacy
import re
import copy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from Levenshtein import *
#nlp = en_core_web_sm.load()
class AuthorInfoExtract():
    STARS=["*",'∗']
    CROSS=["†","‡"]
    def detectCapitalUse(self, word: str) -> bool:
        
        is_upper = [c.isupper() for c in word]          # 大写字符判别列表
        is_lower = [c.islower() for c in word]          # 小写字符判别列表
        
#         if all(is_upper) or all(is_lower):              # 如果所有字符都是大写或小写
#             return True                         

        if any(is_upper):                               # 如果既有大写又有小写
            return is_upper[0] and all(is_lower[1:])    # 要求第一个大写其他都小写
        
    def locateContext(self,resourceId):
        file = open('txt/'+str(resourceId)+'.txt',encoding="utf-8")
        strings=file.read()
        symbolList=["Abstract",'Abstract.','ABSTRACT','ABSTRACT.']
        for symbol in symbolList:
            #if name in strings:
            if strings.find(symbol):
                output=strings[0:strings.find(symbol)]
                break
            else:
                print(symbol+" not found!")
                output==strings[0:3000]
        file.close()
        output=output.replace('\n', ' . ').replace('\r', ' ')
        output=output.split(" ")
        return output
#     def judge(x):
#         return bool(re.search(r'\d', x))

    def symbolJudgement(self,resourceId):
        file = open('txt/'+str(resourceId)+'.txt',encoding="utf-8")
        strings=file.read()
        strings=strings[:3000]
        #print(strings)
        if strings.find("equally"):
            pos= strings.find("equally")
            #print(pos)
            for star in self.STARS:
                if star in strings[pos-50:pos+50]:
                    return 1 #star represent equall 
        if strings.find("Corresponding"):
            pos= strings.find("Corresponding")
            for star in self.STARS:
                if star in strings[pos-50:pos+50]:
                    return 2 #star represent correspinding 
        if "Alphabetical" in strings:
            return 3
        return 0
    
    def matchAuthor(self,roleList,authorList):
        author_role=[]
        for role in roleList:
            highest=0
            index=0
            for i in range(len(authorList)):
                if jaro(role, authorList[i])>highest:
                    index=i
                    highest=jaro(role, authorList[i])
            #print(highest,authorList[index])
            if highest>0.7:
                author_role.append(authorList[index])
        return author_role
    
    def specialAuthors(self,output,authorList):
        starList=[]
        crossList=[]
        for i in range(len(output)):
            for star in self.STARS:
                if star in output[i]:
                    starList.append(" ".join(output[i-2:i+1]))
            for cross in self.CROSS:
                if cross in output[i]:
                    crossList.append(" ".join(output[i-2:i+1]))
        judge=self.symbolJudgement(resourceId)
        #print(judge)
        if judge==0 or judge==1:
            star=self.matchAuthor(starList,authorList)
            corre=self.matchAuthor(crossList,authorList) 
        elif judge==2:
            star=self.matchAuthor(starList,authorList)
            corre=copy.copy(star)
            star=[]
        elif judge==3:
            star=[]
            corre=[]
        #print(star)
        return star,corre

    def sequenceExtratcor(self,output,authorList,parameter=0):
        author_sequence={}
        author_sequence_list=[]
        authorListCopy=copy.copy(authorList)
        num=1
        for i in range(len(output)):
            for name in authorListCopy:
                if output[i] in name and len(output[i])>=2 and self.detectCapitalUse(output[i]):
                    author_sequence[name]=num
                    author_sequence_list.append(name)
                    num+=1
                    authorListCopy.remove(name)
                    break
        if len(author_sequence_list)==len(authorList):
            authorList=author_sequence_list
        if parameter==0:
            return author_sequence
        else:
            return authorList
    
    def main(self,resourceId,authorList,authorPaper,authorCitation,authorID):
        extractor=AuthorInfoExtract()
        
        output=extractor.locateContext(resourceId)
        #print(output)
        author_sequence=extractor.sequenceExtratcor(output,authorList)
        print(author_sequence)
        star,corre=extractor.specialAuthors(output,authorList)
        #print(star)
#         try:
        df=pd.DataFrame.from_dict(author_sequence,orient='index',columns=['sequence'])
        df=df.reset_index().rename(columns={'index':'name'})
        df['star']=0
        for index, row in df.iterrows():
            for author in star:
                if row["name"]==author:
                    df.loc[index,"star"] = 1
        df['corre']=0
        for index, row in df.iterrows():
            for author in corre:
                #print(row["name"],author,index)
                if row["name"]==author:
                    df.loc[index,"corre"] = 1
        df['paperCount']=0
        for index, row in df.iterrows():
            for k,v in authorPaper.items():
                #print(row["name"],author,index)
                if row["name"]==k:
                    df.loc[index,"paperCount"] = v

        df['citationCount']=0
        for index, row in df.iterrows():
            for k,v in authorCitation.items():
                #print(row["name"],author,index)
                if row["name"]==k:
                    df.loc[index,"citationCount"] = v
        df['authorID']=0
        for index, row in df.iterrows():
            for k,v in authorID.items():
                #print(row["name"],author,index)
                if row["name"]==k:
                    df.loc[index,"authorID"] = v
#         except:
#             print("error occured!")
#             df=pd.DataFrame(columns=['name', 'sequence', 'star', 'corre'])
        return df
    
# extractor=AuthorInfoExtract()
# annotation=extractor.main()
# annotation

In [219]:
# def softmax(x):
#     return np.exp(x)/np.sum(np.exp(x),axis=0)
def lookup_id(author,df):
    author_id=""
#     for index, row in df.iterrows():
    for index, row in df.iterrows():
        #print(index)
        #print(k1,v2)
        if author==row["name"]:
            author_id=(df.at[index,"authorID"])
            break
    return author_id
def roleGenerator(df,resourceId):
    ROLES=['Writing_Draft','Writing_Review_Editing','Supervision','Resource','Software']
    LEVELS=[10,50,100,200,1000,3000,5000,10000]
    df_role = pd.DataFrame(0.0, index=df.name, columns=ROLES)
    #sieve1:Sequence
    for index in range(len(df)):
        df_role.loc[df.name[index],"Writing_Draft"] =1/df.sequence[index]
        df_role.loc[df.name[index],"Writing_Review_Editing"] =(df.sequence[index]/len(df.sequence))-0.1
    #Last author is likely to be supervisioner
    
    #sieve2:Annotation
    for index in range(len(df)):
        if df.star[index]==1:
            df_role.loc[df.name[index],"Writing_Draft"] +=1
        if (len(df.star[df.star>0]))<=2:
            if df.corre[index]==1:
                df_role.loc[df.name[index],"Writing_Review_Editing"] +=1
                df_role.loc[df.name[index],"Supervision"] +=0.5
                #df_role.loc[df.name[index],"Supervision"] +=0.5
            
    #sieve3:Citation and Paper
    for index in range(len(df)):
        df_role.Supervision[index]=df.citationCount[index]*0.2/sum(df.citationCount)+df.paperCount[index]*0.8/sum(df.paperCount)
#         print(v)
    for index in range(len(df)):
        num=0.0
        for level in LEVELS:
            if df.citationCount[index]>level:
                num+=0.125
            if df.paperCount[index]>level:
                num+=0.125
        df_role.Resource[index]=num
        df_role.Writing_Review_Editing[index]+=num/5
        
    author_role={}
    paper_role={}
    result_1=[]
    result_2=[]
    try:
        supervisionMax=max(df_role.Supervision)
       # print(supervisionMax)
        for index in range(len(df_role)):
            author_role={}
            author_id=lookup_id(df.name[index],df)
            if df_role.loc[df.name[index],"Writing_Draft"]>=1:
                author_role[author_id] ="Writing_Draft"#isContributionByAuthor
                paper_role[resourceId]="Writing_Draft"#isContributionToPaper
            if df_role.loc[df.name[index],"Writing_Review_Editing"]>=1:
                author_role[author_id] ="Writing_Review_Editing"
                paper_role[resourceId]="Writing_Review_Editing"
            if df_role.loc[df.name[index],"Resource"]>=1:
                author_role[author_id] ="Resource"
                paper_role[resourceId]="Resource"
            if df_role.loc[df.name[index],"Supervision"]==supervisionMax:
                author_role[author_id] ="Supervision"
                paper_role[resourceId]="Supervision"
            if author_role!={}:
                result_1.append(author_role)
                result_2.append(paper_role)
    except:
        print("fail to generate roles")
#def roleClassification(df):
    #print(df_role)
    return result_1,result_2
#     for index, row in df_role.iterrows():
#         for k,v in df.sequence
#             #print(row["name"],author,index)
#             if row["name"]==k:
#                 df_role.loc[index,"citationCount"] = v
#roleGenerator(annotation)


From the generated txt file, first we try to extract the email address by using regular experssion
### TODO: Special format such as {aaa, bbb, ccc}@xxx.com


Secondly we match the author name list with the email list by token matching method (Levenshtein)
### TODO: HOW can we access author list? 

In [200]:
from Levenshtein import *
import re
class EmailInfoExtract():
    def __init__(self):
        self.name_email={}
        self.result=[]
    def get_alpha_str(self,s):
        result = ''.join([x for x in s if x.isalpha()])
        return result

    def email_extraction(self,file_name):
        file = open('txt/'+str(file_name)+'.txt',encoding="utf-8")
        strings=file.read()
        #First page's txt
        strings=strings[:3000]
        matches = []
        matchesGroup=[]
        #1. singular email address
        emailRegex = re.compile(r'''(
            [a-zA-Z0-9._%+-]+      # username
            @+                     # @ symbol

           [a-zA-Z0-9_-]+
           (\.[a-zA-Z0-9_-]+)+
            )''', re.VERBOSE)
        # using RE to match all the patterns in the txt
        for groups in emailRegex.findall(strings):
            matches.append(groups[0])

        list2 = matches
        list_nums = len(list2)
        emailList=[]
        for line in range(list_nums):
            emailList.append(list2[line])
        #2. grouped email list {}
        emailRegex2 = re.compile(r'''(
        \{(.+?)\}+
        @                      # @ symbol

        [a-zA-Z0-9.-]+        # domain name

        (\.[a-zA-Z]{1,4}){1,2} # dot-something
        )''', re.VERBOSE)
        for groups in emailRegex2.findall(strings):
            #print(groups)
            matchesGroup.append(groups[0])
        list3 = matchesGroup
        if len(matchesGroup)>0:
            for gp in list3:
                stringGroup = gp
                index=stringGroup.find("@", 0)
                prefix=stringGroup[:index]
                suffix=stringGroup[index:]
                tempNameList=prefix[1:-1].split(",");
                for name in tempNameList:
                    emailList.append(name+suffix)
                #print(emailList)
        #3. convert to sequenced list if possible 
        sequencedList=[]
        temp_dict={}
        for mail in emailList:
            index=mail.find("@",0)
            prefix=mail[:index]
            email_index=strings.find(prefix)
            temp_dict[mail]=email_index
        sort_idct=sorted(temp_dict.items(),key = lambda x:x[1],reverse = False)
        for k,v in sort_idct:
            sequencedList.append(k)
        #print((sequencedList))

        if len(sequencedList)==len(emailList):
            return sequencedList
        else:
            return emailList
    def lookup_id(self,author,df):
        author_id=""
    #     for index, row in df.iterrows():
        for index, row in df.iterrows():
            #print(index)
            #print(k1,v2)
            if author==row["name"]:
                author_id=(df.at[index,"authorID"])
                break
        return author_id

    def order_matching(self,nameList, emailList):   
        print("ORDER MATCHING")
        for i in range(len(nameList)):
            pat = re.compile(''+'(.*?)'+'@', re.S)
            email = pat.findall(emailList[i])
            email=self.get_alpha_str(email)
            name=nameList[i]
            if (jaro(name, str(email)))<0.4:
                return False
            else:
                self.name_email[name]=email
        return name_email

    #2.No macthes, need to group based on experience and naive approach
    def naive_matching(self,nameList, emailList):
        print("NAIVE MATCHING")
        for name in nameList:
            highest=0
            index=0
            for i in range(len(emailList)):
                #name=filter(lambda x: x.isalpha(), name)
                #using re to extract the first part of email
                pat = re.compile(''+'(.*?)'+'@', re.S)
                email = pat.findall(emailList[i])
                email=self.get_alpha_str(email)
                #print(str(email))
                #print(jaro(name, str(email)))
                #using jaro to calculate the similarity between two strings
                if jaro(name, str(email))>highest:
                    index=i
                    highest=jaro(name, str(email))
            #set pair with the highest score
                    #print(name,highest,emailList[i])
            #self.name_email[name]=emailList[index]
            #If very sure, remove it from the list to reduce the uncertainty for other pairs
            if highest>=0.6:
                self.name_email[name]=emailList[index]
                emailList.remove(emailList[index])
            if highest<=0.4:
                self.name_email[name]=''
        for email in emailList:
            if email not in self.name_email:
                for name in self.name_email:
                    if self.name_email[name]=='':
                        self.name_email[name]=email
                        #print(jaro(name, str(email)))
        return self.name_email
    
    def main(self,resourceId,authorList,df):
        extractor=AuthorInfoExtract()
        output=extractor.locateContext(resourceId)
        nameList=extractor.sequenceExtratcor(output,authorList,1)
        emailList=self.email_extraction(resourceId)
        #print(emailList)
        if len(emailList)==0:
            return
        name_email={}
        #1.If sequenced list and length matches
        if len(nameList)==len(emailList):
            self.name_email=self.order_matching(nameList, emailList)
        if not name_email:
            self.name_email={}
            self.name_email=self.naive_matching(nameList, emailList)
        
        for k,v in self.name_email.items():
            author_id=self.lookup_id(k,df)
            #print(author_id)
            temp={}
            temp[author_id]=v
            self.result.append(temp)
        return self.result
        #return self.name_email
    
# emailExtractor=EmailInfoExtract()
# name_email=emailExtractor.main()

# name_email

In [ ]:
from rdflib import Graph, ConjunctiveGraph, Literal, BNode, Namespace, RDF, URIRef,RDFS
from rdflib.namespace import DC, FOAF
from owlrl import DeductiveClosure, RDFS_Semantics
import pprint
def create_triple_email(totalList):
    g = Graph()
    for lst in totalList:
        if lst!=None:
            for dic in lst:
                for author_id,v1 in dic.items():  
                    #author = BNode()
#                     print(k1,v1)
#                     author_id=lookup_id(k1,df)
#                     print(author_id)
                    author = URIRef("http://www.scoaring.com/ontology#%s")%author_id
                    #print(author)
                    g.add((author, FOAF.mbox, Literal(v1)))
    print(len(g))
    #print(g.serialize(format='turtle').decode())
    #g.serialize(destination='output.txt', format='turtle')
    
def create_triple_role(totalList):
    g = Graph()
    for lst in totalList:
        if lst!=None:
            for dic in lst:
                for author_id,v1 in dic.items():
                    author = URIRef("http://www.scoaring.com/ontology#%s")%author_id
                    predict= URIRef(http://www.scoaring.com/ontology)#refersTo
                    g.add((author, FOAF.mbox, Literal(v1)))
create_triple(totalList)

In [220]:
def main():
    processing=Processing()
    processing.NUM=5
    email=[]
    role_1=[]
    role_2=[]
    for i in range(processing.NUM):
        try:
            resourceId,authorList,authorPaper,authorCitation,authorID=processing.main(i)
            #print(authorList)
            extractor=AuthorInfoExtract()
            annotation=extractor.main(resourceId,authorList,authorPaper,authorCitation,authorID)
            #print(annotation)
            author_role,paper_role=roleGenerator(annotation,resourceId)
            role_1.append(author_role)
            role_2.append(paper_role)
            #print(author_role)
            emailExtractor=EmailInfoExtract()
            name_email=emailExtractor.main(resourceId,authorList,annotation)
            email.append(name_email)
        except:
            continue
    file = open("email_output2.txt", "w")
    file.write(str(email))
    file = open("role_output2.txt", "w")
    file.write(str(role_1))
    file.write(str(role_2))
main()

P1156351 http://arxiv.org/pdf/1710.01269v1.pdf
{'Christian S. Perone': 1, 'Evan Calabrese': 2, 'Julien Cohen-Adad': 3}
P1156365 http://arxiv.org/pdf/1803.03095v1.pdf
{'Xialei Liu': 1, 'Joost van de Weijer': 2, 'Andrew D. Bagdanov': 3}
ORDER MATCHING
NAIVE MATCHING
P1156386 http://arxiv.org/pdf/1706.03686v3.pdf
{'Kang Han': 1, 'Wanggen Wan': 2, 'Haiyan Yao': 3, 'Li Hou': 4}
NAIVE MATCHING
P1156541 https://arxiv.org/pdf/1806.06371v2.pdf
{'Lisa Beinborn': 1, 'Teresa Botschen': 2, 'Iryna Gurevych': 3}
P1156614 http://arxiv.org/pdf/1709.04496v2.pdf
{'Christian F. Baumgartner': 1, 'Lisa M. Koch': 2, 'Marc Pollefeys': 3, 'Ender Konukoglu': 4}


SyntaxError: invalid syntax (<ipython-input-84-d5c2340c4b2d>, line 11)